# Word2vec context nearest neighbour model

In [318]:
#package to load word2vec vectors
import gensim
from gensim.models.keyedvectors import KeyedVectors
#self built functions
import utilities
#semcor corpus
import nltk
from nltk.corpus import semcor
from nltk.corpus import wordnet as wn
#Micellaneous
import numpy as np
import collections
from scipy.spatial.distance import cosine
import sys
import re
import time
import progressbar
import pickle
from functools import partial

In [184]:
#Load word2vec binary file
model = KeyedVectors.load_word2vec_format('../datasets/word2vec/GoogleNews-vectors-negative300.bin', binary=True)
#Define embedding_dict
embedding_dict = model.word_vec
#Load semcor
tagged_chunks = semcor.tagged_chunks(tag='sem')

In [207]:
#Progress bar utilities
class ProgressBar(object):
    DEFAULT = 'Progress: %(bar)s %(percent)3d%%'
    FULL = '%(bar)s %(current)d/%(total)d (%(percent)3d%%) %(remaining)d to go'

    def __init__(self, total, width=40, fmt=DEFAULT, symbol='=',
                 output=sys.stderr):
        assert len(symbol) == 1

        self.total = total
        self.width = width
        self.symbol = symbol
        self.output = output
        self.fmt = re.sub(r'(?P<name>%\(.+?\))d',
            r'\g<name>%dd' % len(str(total)), fmt)

        self.current = 0

    def __call__(self):
        percent = self.current / float(self.total)
        size = int(self.width * percent)
        remaining = self.total - self.current
        bar = '[' + self.symbol * size + ' ' * (self.width - size) + ']'

        args = {
            'total': self.total,
            'bar': bar,
            'current': self.current,
            'percent': percent * 100,
            'remaining': remaining
        }
        print('\r' + self.fmt % args, file=self.output, end='')

    def done(self):
        self.current = self.total
        self()
        print('', file=self.output)

In [325]:
#prediction method
def predict(context,predict_lemmas):
    #senses_choices must be a python list of sense label (wordnet synset)
    #get context embedding
    context_emb = getContextEmb_word2vec(context=context,emb_size=300,embedding_dict=embedding_dict)
    #get senses choice
    synsets = wn.synsets(predict_lemmas)
    senses_choices = [synset.name() for synset in synsets]
    #calculate cosine distance between each sense and context
    decision_chart = [[choice,cosine(context_emb,sense_embeddings[choice])] for choice in senses_choices]
    prediction = sorted(decision_chart,key=lambda x:x[1])[0][0]
    return prediction
def get_context(tagged_chunks,position,window_size):
    center = tagged_chunks[position].leaves()
    num_words = 0
    iter_position = position
    #unroll left size
    left = []
    right = []
    while (num_words<window_size):
        iter_position-=1
        if iter_position <0:
            break
        item = tagged_chunks[iter_position]
        if type(item)==list:
            num_words +=1
            left.insert(0,item[0])
        elif type(item)==nltk.tree.Tree:
            words2append = item.leaves()
            if num_words+len(words2append)>window_size:
                num_allowed = window_size-num_words
                words2append=words2append[-num_allowed:]
            num_words += len(words2append)
            left = words2append+left
            
    num_words = 0
    iter_position = position
    while (num_words<window_size):
        iter_position+=1
        if iter_position >=len(tagged_chunks):
            break
        item = tagged_chunks[iter_position]
        if type(item)==list:
            num_words +=1
            right.append(item[0])
        elif type(item)==nltk.tree.Tree:
            words2append = item.leaves()
            if num_words+len(words2append)>window_size:
                num_allowed = window_size-num_words
                words2append=words2append[:num_allowed]
            num_words += len(words2append)
            right = right+words2append
    return left+center+right

def getContextEmb_word2vec(context,embedding_dict,emb_size,unk_emb=np.zeros(300)):
    # Input introductions
    # sentence: an array of tokens of untagged sentence. 
    # center: position of the center word
    # window_size: size of context window
    # embedding_Dict: gensim model method
    ################################################################
        output_embedding = np.zeros(emb_size)
        for word in context:
            try:
                output_embedding+=embedding_dict(word)#use gensim model method
            except:
                output_embedding+=unk_emb
        return output_embedding
    
def buildSemEmb_word2vec(tagged_chunks,embedding_dict,emb_size=300,window_size=4):
    progress = progressbar.ProgressBar(max_value=len(tagged_chunks))
    output_dict = collections.defaultdict(partial(np.zeros,emb_size))
    for idx in range(len(tagged_chunks)):
        progress.update(idx)
        itm=tagged_chunks[idx]
        if(type(itm))==list:
            continue
        else:
            #Use try except handling since some of the label is broken
            try:
                sense_index = itm.label().synset().name()
            except:
                continue
            context = get_context(position=idx,tagged_chunks=tagged_chunks,window_size=window_size)
            context_emb = getContextEmb_word2vec(context,embedding_dict=embedding_dict,emb_size=300)
            output_dict[sense_index]+=context_emb
    return output_dict

In [210]:
#build sense embeddings
sense_embeddings = buildSemEmb_word2vec(tagged_chunks=tagged_chunks,embedding_dict=embedding_dict)

[========================================] 778587/778587 (100%)      0 to go


In [326]:
sense_mebeddings_win5 = buildSemEmb_word2vec(tagged_chunks=tagged_chunks,embedding_dict=embedding_dict,window_size=5)

 99% (778332 of 778587) |################# | Elapsed Time: 0:16:06 ETA: 0:00:00

In [328]:
f = open('sense_win5.pk','wb')
pickle.dump(obj=sense_mebeddings_win5,file=f)

In [329]:
f.close()

# Attempt: trying to classify some ambiguous word

In [285]:
#Example test
example = chunks[:30]
context = get_context(position=5,tagged_chunks=example,window_size=5)
print('lemma to predict:%s'%(example[5]))
print('context:%s'%(' '.join(context)))
senses_choices = example[5]
print()
print('Final decision: %s'%(predict(context=context,predict_lemmas='investigation')))

lemma to predict:(Lemma('probe.n.01.investigation') investigation)
context:Grand Jury said Friday an investigation of Atlanta 's recent primary

Final decision: probe.n.01


## Notice how closed possible senses are

In [259]:
print(wn.synsets('investigation')[0])
print(wn.synsets('investigation')[0].definition())

Synset('probe.n.01')
an inquiry into unfamiliar or questionable activities


In [260]:
print(wn.synsets('investigation')[1])
print(wn.synsets('investigation')[1].definition())

Synset('investigation.n.02')
the work of inquiring into something thoroughly and systematically


In [281]:
lemma = example[10]
lemma.label().name()

'primary_election'

# Perform all-words WSD

In [327]:
def predict_all(tagged_chunks,window_size = 4):
    progress = progressbar.ProgressBar(max_value=len(tagged_chunks))
    output_dict = collections.defaultdict(lambda: np.zeros(emb_size))
    #count_dict = collections.defaultdict(lambda: 0)
    num_correct = 0
    num_predicted = 0.0
    for idx in range(len(tagged_chunks)):
        progress.update(idx)
        itm=tagged_chunks[idx]
        if (idx%100000==0)&(num_predicted>0):
            print('correct: %s, predicted: %s, accuracy: %s'%(num_correct,num_predicted,num_correct/num_predicted))
        if(type(itm))==list:
            continue
        else:
            #Use try except handling since some of the label is broken
            try:
                lemma = itm.label().name()
            except:
                continue
            context = get_context(position=idx,tagged_chunks=tagged_chunks,window_size=window_size)
            prediction = predict(context=context,predict_lemmas=lemma)
            correct = itm.label().synset().name()
            num_predicted +=1
            if prediction == correct:
                num_correct+=1
    return num_correct/num_predicted

In [297]:
# Window size 4 should get 68.3% accuracy.
predict_all(tagged_chunks,window_size=4)

[                                        ]  10069/778587 (  1%) 768518 to go

correct: 3669, predicted: 4822.0, accuracy: 0.7608875985068436


[=                                       ]  20046/778587 (  2%) 758541 to go

correct: 7121, predicted: 9430.0, accuracy: 0.7551431601272535


[=                                       ]  30045/778587 (  3%) 748542 to go

...

correct: 152346, predicted: 222304.0, accuracy: 0.6853048078307183


[======================================= ] 770362/778587 ( 98%)   8225 to go

correct: 152870, predicted: 223624.0, accuracy: 0.6836028333273709


[========================================] 778587/778587 (100%)      0 to go


NameError: name 'num_predict' is not defined